In [1]:
import pandas as pd

In [2]:
df_menu = pd.read_excel('./detail.xlsx', header=0).fillna('')
list_vehicle_url = df_menu['vehicle_url'].to_list()

print('总数量：' + str(len(list_vehicle_url)))
list_vehicle_url

总数量：150


['https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76398&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76754&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=66592&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=72789&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106223&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131698&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=53418&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=65302&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=82894&PartType=HVAC%20Control%20Module',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=128361&Pa

In [3]:
df = pd.DataFrame()

df

""


In [4]:
from selenium.webdriver import ChromeOptions

In [5]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
# option.add_argument('--proxy-server=')
# option.add_argument('user-agent=')
option.page_load_strategy = 'eager'

In [6]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

In [7]:
browser = Chrome(service=Service('../../../chromedriver'), options=option)

browser.maximize_window()

In [8]:
import time

i = 0

browser.get(list_vehicle_url[i])

time.sleep(3)

In [9]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

In [10]:
page = 0

dict_vehicle = {}
while True:
    print(page)
    wait = WebDriverWait(browser, 60)
    wait.until(EC.presence_of_element_located((By.XPATH, '//tr[@class="detail-app-row"]')))
    list_row = browser.find_elements(by=By.XPATH, value='//tr[@class="detail-app-row"]')
    
    for row in list_row:
        make = row.find_element(by=By.XPATH, value='./td[2]').text.strip()
        model = row.find_element(by=By.XPATH, value='./td[3]').text.strip()
        year = row.find_element(by=By.XPATH, value='./td[1]').text.strip()

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: [year]}
        else:
            if model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            else:
                dict_vehicle[make][model].append(year)
                
    list_next = browser.find_elements(by=By.XPATH, value='//a[@id="pagingBottom_nextButton"]')
    if len(list_next) == 0:
        break
    else:
        list_next[0].click()
        
    page += 1
    
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_year = list(set(dict_vehicle[make][model]))
        if len(list_year) == 1:
            list_vehicle.append(make + ' ' + model + ' ' + list_year[0])
        else:
            list_vehicle.append(make + ' ' + model + ' ' + min(list_year) + '-' + max(list_year))

vehicle = '\n'.join(list_vehicle)

print()
print(vehicle)

0

Chrysler 300 2005-2007
Dodge Charger 2006-2007
Dodge Magnum 2005-2007


In [11]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'vehicle_url': list_vehicle_url[i],
                         'Vehicle': vehicle}])

browser.quit()

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,status,vehicle_url,Vehicle
0,ok,https://www.dormanproducts.com/itemdetailapp.a...,Chrysler 300 2005-2007\nDodge Charger 2006-200...
